# **Vectorización de los Chunks Generados**

En este notebook vamos a vectorizar (convertir en embeddings) a los chunks que hemos guardado en el archivo corpus.json generado por el script pipene_data.py, usaremos SentenceTransformer con un modelo pre-entrenado con varios idiomas incluido el español.

Para entender mejor el objetivo de lo que haremos es ubicar cada chunk en un espacio n-dimensional de manera que el modelo pueda identificar la relación semantica que hay entre los chunks ¿Como asi? haremos un ejemplo sencillo:

- Un chunk que habla de "reposición por despido" estará muy cerca de otro que mencione "reincorporación de un trabajador".
- Un chunk que hable de "régimen laboral privado" estará lejos de uno que hable de "derecho penal".

Al convertir estos chunks de textos a embeddings la computadora puede "entender" la similitud que tienen estos chunks a traves del angulo que hacen estos vectores en el espacio muldimensional. 

In [26]:
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os

In [27]:
# Cargamos el corpus
with open("../src/corpus.json", 'r', encoding='utf-8') as f:
    corpus = json.load(f)

Para esta primera muestra, vamos a trabajar con una muestra de 10 PDF's del total de nuestro corpus

In [28]:
len(corpus)

10

En cada muestra tenemos el nombre del archivo, los metadatos y los chunks.

In [29]:
corpus[0].keys()

dict_keys(['archivo', 'metadatos', 'chunks'])

En cada metadatos hemos extraido el numero de casacion, el lugar, y la materia de la sentencia

In [30]:
corpus[0]['metadatos'].keys()


dict_keys(['n_casacion', 'lugar', 'materia'])

En cada chunk de una sentencia extrajimos información como la sumilla, vista, materia, desicion

In [31]:
for chunk in corpus[0]['chunks']:
    print("\n" +chunk[:30] + "...")


CORTE SUPREMA DE JUSTICIA DE L...

Sumilla. Es en ese contexto do...

VISTA la causa número dieciséi...

MATERIA DEL RECURSO 
Se trata ...

CAUSAL DEL RECURSO 
El recurso...

CONSIDERANDO 
Desarrollo del p...

DECISIÓN 
Declararon FUNDADO e...


Usaremos un modelo preentrenado con multiples lenguajes incluso el español:

In [32]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Ahora vamos a iterar sobre cada uno de los documentos de nuestro corpus para generar los embedings, y luego añadimos el embedding a cada chunk.

In [33]:
for document in tqdm(corpus, desc="Vectorizando documentos"):
    
    chunk_texts = [chunk for chunk in document['chunks']]
    
    embeddings = model.encode(chunk_texts, show_progress_bar=False)
    
    enriched_chunks = []
    for i, chunk_text in enumerate(chunk_texts):
        enriched_chunks.append({
            "text": chunk_text,
            "embedding": embeddings[i].tolist()     
        })
    
    document['chunks'] = enriched_chunks

Vectorizando documentos: 100%|██████████| 10/10 [00:03<00:00,  3.18it/s]


In [38]:
document['chunks'][0]["embedding"][:10]

[-0.009157266467809677,
 -0.015870476141572,
 -0.01508508063852787,
 -0.04422919452190399,
 -0.12381163984537125,
 0.009335299022495747,
 0.04907502979040146,
 -0.058677442371845245,
 -0.05796046555042267,
 -0.0022972796577960253]

In [39]:
output_path = "../output/corpus_vectorizado.json"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

print(f"Guardando corpus vectorizado en: {output_path}")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(corpus, f, indent=2, ensure_ascii=False)

Guardando corpus vectorizado en: ../output/corpus_vectorizado.json
